In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Definir la carpeta donde se guardarán los archivos HTML
folder_path = "C:\Users\kevin\OneDrive\Documentos\Deberes RI\ir24a\week12\data"

In [3]:
# URL de la página principal de recetas
base_url = "https://www.allrecipes.com/"

In [4]:
# Función para descargar y guardar una receta
def download_recipe(url, folder_path):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        recipe_title = soup.find("meta", {"property": "og:title"})['content'].replace(" ", "_").replace("/", "_")
        file_path = os.path.join(folder_path, f"{recipe_title}.html")
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(soup.prettify())

In [5]:
# Obtener el contenido de la página principal
response = requests.get(base_url)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    # Encontrar todos los enlaces a recetas
    recipe_links = soup.find_all('a', class_='tout__titleLink')

    for link in recipe_links:
        recipe_url = link['href']
        download_recipe(recipe_url, folder_path)


In [6]:
print("Descarga de recetas completada.")

Descarga de recetas completada.


In [7]:
# Lista para almacenar los datos de cada archivo
data_list = []

In [8]:
# Iterar sobre todos los archivos en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith(".html"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        # Parsear el contenido HTML
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extraer título
        title = soup.find("meta", {"property": "og:title"})['content'] if soup.find("meta", {"property": "og:title"}) else "No Title"

        # Extraer descripción
        description = soup.find('p', class_='article-subheading type--dog').text if soup.find('p', class_='article-subheading type--dog') else "No Description"

        # Extraer pasos
        steps = [item.text.strip() for item in soup.find_all("p", class_='comp mntl-sc-block mntl-sc-block-html')]

        # Extraer ingredientes
        ingredients = [item.text.strip() for item in soup.find_all("li", class_='mm-recipes-structured-ingredients__list-item')]
        
        # Agregar los datos a la lista
        data_list.append({
            "Title": title,
            "Description": description,
            "Steps": steps,
            "Ingredients": ingredients
        })

In [9]:
# Crear un DataFrame con los datos de todos los archivos
df = pd.DataFrame(data_list)

# La pagina bloquea el acceso al intentar realizarlo 